### 반복해서 simulation point를 만드는 script 구현

 - 입력 : STL points, vertex, plane, normal vector,40 point (.npy), number of iter. (int)
 - 출력 : txt * number of iter.

In [1]:
import math
import numpy as np
import csv
import random
from sklearn.neighbors import KDTree

In [15]:
def select_random_vertex_index(point40_on_STL, vertex_data):
    """
    input data structure
    points = [[x0, y0, z0], ... [xn, yn, zn]]
    vertex_data = [[[px00, py00, pz00], [px01, py01, pz01], [px02, py02, pz02]],
                    ...,
                   [[pxm0, pym0, pzm0], [pxm1, pym1, pzm1], [pxm2, pym2, pzm2]]]
                   
    output data structure
    selected_index = [i0, i1, ... in]
    """
    
    vertex_group                  = []

    for i in range(len(point40_on_STL)):
        temp                      = []
        for j in range(len(vertex_data)):
            if point40_on_STL[i] in vertex_data[j]:
                temp.append(j)
        vertex_group.append(temp)

    vertex_selected_index         = []

    for i in range(len(vertex_group)):
        randomIndex               = random.choice(vertex_group[i])
        vertex_selected_index.append(randomIndex)
        
    return vertex_selected_index, vertex_group

def random_vector(length = 1):
    """
    make vector that have length in sphere
    
    input data structure
    vector length = int
    
    output data structure
    vector = [vx, vy, vz]
    """
    
    rand_i, rand_j = np.random.rand(2)            # Two independent random numbers from a uniform distribution in the range (0, 1)
    theta          = 2 * np.pi * rand_i           # Spherical coordinate theta
    phi            = np.arccos(2 * rand_j - 1)    # Spherical coordinate phi, corrected for distribution bias
    x              = np.cos(theta) * np.sin(phi)  # Cartesian coordinate x
    y              = np.sin(theta) * np.sin(phi)  # Cartesian coordinate y
    z              = np.cos(phi)                  # Cartesian coordinate z
    vector         = [x*math.sqrt(length), y*math.sqrt(length), z*math.sqrt(length)]

    return vector

def add_mm_error_to_point(point, max_error = 20):
    """
    input data structure
    point = [x, y, z] # x, y, z unit = mm
    
    output data structure
    point_added_error = [x', y', z']
    """
    
    point_added_error      = point.copy()
    
    error                  = random.uniform(0, max_error)
    
    xyz_error              = random_vector(error)

    point_added_error[0]   += xyz_error[0]
    point_added_error[1]   += xyz_error[1]
    point_added_error[2]   += xyz_error[2]
    
    return point_added_error

def find_projection_point_in_plane_3D(point, plane, norm_vect):
    """
    input data structure
    point = [x, y, z]
    plane = [a, b, c, d] # plane eq. = ax + by + cz + d = 0
    norm_vect = [i, j, k]
    
    output data structure
    True -> projected_point = [x', y', z']
    False -> value of diff. with plane = float(d)
    """
    
    plane_ABC_sqrt       = math.sqrt(plane[0]*plane[0] + plane[1]*plane[1] + plane[2]*plane[2])
    dist                 = (plane[3] + plane[0] * point[0] + plane[1] * point[1] + plane[2] * point[2]) / plane_ABC_sqrt
    # print(dist)
    projected_point      = point - dist * norm_vect
    
    on_the_plane_value   = is_including_point(plane, projected_point)
    
    if on_the_plane_value:
        pass
    else:
        print(projected_point)
        # print(on_the_plane_value)
     
    return projected_point 

def is_including_point(plane, point):
    """
    input data structure
    point = [x, y, z]
    plane = [a, b, c, d] # plane eq. = ax + by + cz + d = 0
    
    output data structure
    is point included in plane = bool
    """
    
    a                 = plane[0]
    b                 = plane[1]
    c                 = plane[2]
    d                 = plane[3]
    x                 = point[0]
    y                 = point[1]
    z                 = point[2]
    
    included_value    = a*x + b*y + c*z + d
    # print(included_value)
    
    if included_value < 0.00001:
        return True
    else:
        return False
    
def cal_points40_with_error_on_plane(point40_on_STL, plane_selected, vector_selected):
    """
    input data structure
    points = [[xr0, yr0, zr0], ... [xrn, yrn, zrn]]
    plane_data = [[a0, b0, c0, d0], [a1, b1, c1, d1], ... [am, bm, cm, dm]]
    vector_data = [[vx0, vy0, vz0], ... [vxn, vyn, vzn]]
    
    output data structure (closet points in trg_points from ref_points)
    points_on_plane = [[xpr0, ypr0, zpr0], ... [xprn, yprn, zprn]]
    """
    
    points40_with_error_on_plane    = []

    data_point_40_slt               = point40_on_STL.copy()

    for i in range(len(data_point_40_slt)):        
        _point_added_error          = add_mm_error_to_point(data_point_40_slt[i])
        _point_on_plane             = find_projection_point_in_plane_3D(_point_added_error, 
                                                                        plane_selected[i],
                                                                        vector_selected[i])
        
        if np.isnan(np.sum(_point_on_plane)):
            points40_with_error_on_plane.append(data_point_40_slt[i])
        else:
            points40_with_error_on_plane.append(_point_on_plane)
            
    return points40_with_error_on_plane

def get_dist_btw_2_3Dpoints(point1, point2):
    """
    input data structure
    point1 = [x1, y1, z1]
    point2 = [x2, y2, z2]
    
    output data structure
    distance
    """
    
    sum        = 0
    for i in range(3):
        sum    = sum + (point1[i] - point2[i])**2
    return math.sqrt(sum)

def get_closet_points(ref_points, trg_points):
    """
    input data structure
    ref_points = [[xr0, yr0, zr0], ... [xrn, yrn, zrn]]
    trg_points = [[xt0, yt0, zt0], ... [xtm, ytm, ztm]]
    
    output data structure (closet points in trg_points from ref_points)
    closet_points = [[xc0, yc0, zc0], ... [xcn, ycn, zcn]]
    """
    
    tree                         = KDTree(trg_points)
    nearest_dist, nearest_ind    = tree.query(ref_points, k=2)
    
    nearest_index                = nearest_ind[:,1]
    closet_points                = []
    
    for i in range(len(nearest_index)):
        closet_points.append(trg_points[nearest_index[i]])
        
    return closet_points
    
def get_data_with_index(index_data, some_data):
    """
    input data structure
    index_data = [i0, i1, ... in]
    some_data = plane_data or vertex_data or vector_data
    plane_data = [[a0, b0, c0, d0], [a1, b1, c1, d1], ... [am, bm, cm, dm]]
    vertex_data = [[[px00, py00, pz00], [px01, py01, pz01], [px02, py02, pz02]],
                    ...,
                   [[pxm0, pym0, pzm0], [pxm1, pym1, pzm1], [pxm2, pym2, pzm2]]]
    vector_data = [[vx0, vy0, vz0], ... [vxm, vym, vzm]]
    
    output data structure
    some_data_indexed = plane_data or vertex_data or vector_data
    plane_data = [[a0, b0, c0, d0], [a1, b1, c1, d1], ... [an, bn, cn, dn]]
    vertex_data = [[[px00, py00, pz00], [px01, py01, pz01], [px02, py02, pz02]],
                    ...,
                   [[pxn0, pyn0, pzn0], [pxn1, pyn1, pzn1], [pxn2, pyn2, pzn2]]]
    vector_data = [[vx0, vy0, vz0], ... [vxn, vyn, vzn]]
    """
    
    some_data_indexed = []
    
    for i in range(len(index_data)):
        some_data_indexed.append(some_data[index_data[i]])
        
    return some_data_indexed
    
def add_error_to_point_following_to_vector_dirct(point, error_length, vector):
    """
    input data structure
    point = [x, y, z]
    error_length = float (int)
    vector = [unit x vct, unit y vct, unit z vct]
    
    output data structure
    point_added_error_to_vector_direction = [xe, ye, ze]
    """
    
    point_with_error   = []
    unit_error         = []
    
    for i in range(len(point)):
        unit_error.append(error_length * vector[i])
    
    for i in range(len(point)):
        point_with_error.append(point[i] + unit_error[i])
    
    return point_with_error

In [4]:
num_of_iter = 10

num_of_files = 8

init_points_data_path = 'C:/Users/eornr/Desktop/source_crx/200331_point_gen/01_point_gen_in_STL/05_gen_point_init_planing_data/data'

STL_info_data_path = 'C:/Users/eornr/Desktop/source_crx/200331_point_gen/01_point_gen_in_STL/01_STL_data_to_npy/data'
info_data_prefix = ['/Fc1L', '/Tc1L', '/Fc1R', '/Tc1R',
                    '/Fc2L', '/Tc2L', '/Fc2R', '/Tc2R']

point40_data_path = 'C:/Users/eornr/Desktop/source_crx/200331_point_gen/01_point_gen_in_STL/02_raw_40_point/data'

point_processed_data_path = 'C:/Users/eornr/Desktop/source_crx/200331_point_gen/01_point_gen_in_STL/06_gen_point_data_proto/data'

In [35]:
# !!!
test_nof = 0

# check file name
file_name = point_processed_data_path + info_data_prefix[test_nof] + '_' + str(0) + '_points_processed.txt'
print(file_name)

# load raw data
point_init_plan = np.load(init_points_data_path + info_data_prefix[test_nof] + '_plan.npy')
point40_on_STL = np.load(point40_data_path + info_data_prefix[test_nof] + 'points_40_selected.npy')
each_point_data = np.load(STL_info_data_path + info_data_prefix[test_nof] + '_femur_each_point_data.npy')
vertex_data = np.load(STL_info_data_path + info_data_prefix[test_nof] + '_femur_vertex_data.npy')
plane_data = np.load(STL_info_data_path + info_data_prefix[test_nof] + '_femur_plane_data.npy')
vector_data = np.load(STL_info_data_path + info_data_prefix[test_nof] + '_femur_vector_data.npy')

C:/Users/eornr/Desktop/source_crx/200331_point_gen/01_point_gen_in_STL/06_gen_point_data_proto/data/Fc1L_0_points_processed.txt


###### gen point init PLAN probing

In [7]:
# select planer init point (on bone model)
point_init_plan_on_bone = get_closet_points(point_init_plan, each_point_data)

In [8]:
# gen point
select_vertex_index_raw_init, _ = select_random_vertex_index(point_init_plan_on_bone, vertex_data)

In [16]:
point_init_with_error_on_plane = cal_points40_with_error_on_plane(point_init_plan_on_bone, 
                                                                  get_data_with_index(select_vertex_index_raw_init, plane_data), 
                                                                  get_data_with_index(select_vertex_index_raw_init, vector_data)) 

In [17]:
nearest_point_init_with_error_on_plane = get_closet_points(point_init_with_error_on_plane, each_point_data)

In [18]:
select_vertex_index_nearest_init, _ = select_random_vertex_index(nearest_point_init_with_error_on_plane, vertex_data)

In [19]:
nearest_point_init_vector = get_data_with_index(select_vertex_index_nearest_init, vector_data)

In [64]:
point_init_with_uncertainty_error     = []

for i in range(len(nearest_point_init_with_error_on_plane)):
    error_value                       = random.uniform(-0.7, 0.7)
    tmp_point                         = add_error_to_point_following_to_vector_dirct(nearest_point_init_with_error_on_plane[i],
                                                                                     error_value,
                                                                                     nearest_point_init_vector[i])

    point_init_with_uncertainty_error.append(tmp_point)

In [65]:
len(point_init_with_uncertainty_error)

5

###### gen point ICP probing

In [23]:
# gen point
select_vertex_index_raw, _ = select_random_vertex_index(point40_on_STL, vertex_data)

In [24]:
points40_with_error_on_plane = cal_points40_with_error_on_plane(point40_on_STL, 
                                                                get_data_with_index(select_vertex_index_raw, plane_data), 
                                                                get_data_with_index(select_vertex_index_raw, vector_data)) 

[nan nan nan]


c:\users\eornr\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in double_scalars


In [25]:
nearest_points40_with_error_on_plane = get_closet_points(points40_with_error_on_plane, each_point_data)

In [26]:
select_vertex_index_nearest, _ = select_random_vertex_index(nearest_points40_with_error_on_plane, vertex_data)

In [27]:
nearest_points40_vector = get_data_with_index(select_vertex_index_nearest, vector_data)

In [66]:
points40_with_uncertainty_error     = []

for i in range(len(nearest_points40_with_error_on_plane)):
    error_value                     = random.uniform(-0.7, 0.7)
    tmp_point                       = add_error_to_point_following_to_vector_dirct(nearest_points40_with_error_on_plane[i],
                                                                                   error_value,
                                                                                   nearest_points40_vector[i])

    points40_with_uncertainty_error.append(tmp_point)

In [67]:
points400_with_uncertainty_error = []

for i in range(len(points40_with_uncertainty_error)):
    point = points40_with_uncertainty_error[i]
    for j in range(10):
        points400_with_uncertainty_error.append(add_mm_error_to_point(point, 0.5))

In [68]:
len(points400_with_uncertainty_error)

400

###### save data

In [69]:
regi_data_points = point_init_with_uncertainty_error + points400_with_uncertainty_error

In [70]:
with open(file_name, 'w') as file:
    for i in range(len(regi_data_points)):
        tmp_point = regi_data_points[i]
        tmp_data_str = ""
        for j in range(len(tmp_point)):
            tmp_data_str += str(tmp_point[j])
            if j < 2:
                tmp_data_str += ', '
            else:
                tmp_data_str += '\n'
        file.write(tmp_data_str)

###### save data with transform

In [79]:
def eulerDegree2mtx(data):
    posX = data[0]
    posY = data[1]
    posZ = data[2]
    pitch = data[3] / 180 * 3.141592653589793238462643383279
    roll = data[4] / 180 * 3.141592653589793238462643383279
    yaw = data[5] / 180 * 3.141592653589793238462643383279
    
    Rx = np.array([[1, 0, 0, 0],
                   [0, math.cos(pitch), -1 * math.sin(pitch), 0],
                   [0, math.sin(pitch), math.cos(pitch), 0],
                   [0, 0, 0, 1]])
    
    Ry = np.array([[math.cos(roll), 0, math.sin(roll), 0],
                   [0, 1, 0, 0],
                   [-1 * math.sin(roll), 0, math.cos(roll), 0],
                   [0, 0, 0, 1]])
    
    Rz = np.array([[math.cos(yaw), -1 * math.sin(yaw), 0, 0],
                   [math.sin(yaw), math.cos(yaw), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    
    mtxR = np.dot(Rz, np.dot(Ry, Rx))
    
    mtxR[0][3] = posX
    mtxR[1][3] = posY
    mtxR[2][3] = posZ
    
    return mtxR

def point2mtx(point):
    mtx = np.array([[1, 0, 0, point[0]],
                    [0, 1, 0, point[1]],
                    [0, 0, 1, point[2]],
                    [0, 0, 0, 1       ]])
    
    return mtx

In [72]:
# left
transform_data_trans = [625, 195, 225]
transform_data_euler_degree_rot = [0, 0, 0, 220, 0, 180]
transform_data_mtx = eulerDegree2mtx(transform_data_euler_degree_rot)

regi_points_trans = []

for i in range(len(regi_data_points)):
    tmp_mtx = np.dot(transform_data_mtx, point2mtx(regi_data_points[i]))
    regi_points_trans.append([tmp_mtx[0][3] + transform_data_trans[0], 
                              tmp_mtx[1][3] + transform_data_trans[1], 
                              tmp_mtx[2][3] + transform_data_trans[2]])

In [80]:
# left
transform_data_euler_degree_rot = [625, 195, 225, 220, 0, 180]
transform_data_mtx = eulerDegree2mtx(transform_data_euler_degree_rot)

regi_points_trans = []

for i in range(len(regi_data_points)):
    tmp_mtx = np.dot(transform_data_mtx, point2mtx(regi_data_points[i]))
    regi_points_trans.append([tmp_mtx[0][3], 
                              tmp_mtx[1][3], 
                              tmp_mtx[2][3]])

In [81]:
file_name_ = point_processed_data_path + info_data_prefix[test_nof] + '_' + str(0) + '_points_processed_trns__.txt'

with open(file_name_, 'w') as file:
    for i in range(len(regi_points_trans)):
        tmp_point = regi_points_trans[i]
        tmp_data_str = ""
        for j in range(len(tmp_point)):
            tmp_data_str += str(tmp_point[j])
            if j < 2:
                tmp_data_str += ', '
            else:
                tmp_data_str += '\n'
        file.write(tmp_data_str)

In [74]:
transform_data_mtx

array([[-1.00000000e+00,  9.38133875e-17, -7.87187789e-17,
         0.00000000e+00],
       [ 1.22464680e-16,  7.66044443e-01, -6.42787610e-01,
         0.00000000e+00],
       [ 0.00000000e+00, -6.42787610e-01, -7.66044443e-01,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [59]:
transform_data_mtx

array([[-1.00000000e+00,  9.38133875e-17, -7.87187789e-17,
         6.25000000e+02],
       [ 1.22464680e-16,  7.66044443e-01, -6.42787610e-01,
         1.95000000e+02],
       [ 0.00000000e+00, -6.42787610e-01, -7.66044443e-01,
         2.25000000e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [60]:
regi_points_trans_back = []

for i in range(len(regi_points_trans)):
    tmp_mtx = np.dot(np.linalg.inv(transform_data_mtx), point2mtx(regi_points_trans[i]))
    regi_points_trans_back.append([tmp_mtx[0][3], 
                                   tmp_mtx[1][3], 
                                   tmp_mtx[2][3]])

In [63]:
regi_points_trans

[[41.81985639779847, 13.836211065341958, -11.358150379240156],
 [-49.23404160988244, 12.11141174860605, -10.002401780101739],
 [-6.363323362004559, 11.744469192470977, 11.328449595117748],
 [-9.216402345385555, -19.486665701518813, 15.278925458952768],
 [-3.6864067415884803, 24.701623740689342, -2.597663629164627],
 [33.48091023519138, 40.75628708448377, -1.4633150788463176],
 [33.146334291893865, 40.921765054599945, -1.3809234454335808],
 [33.12314508493693, 41.20235364485768, -1.7941724864132205],
 [33.19953427662369, 40.71114453067558, -1.4680372660744005],
 [33.33408566258632, 41.15301857444272, -1.2406068902495129],
 [33.38932727944583, 40.66200184453432, -1.4738192066590408],
 [33.93083863538277, 41.00715175331585, -1.3496650625588202],
 [33.431032837174236, 40.745848460035546, -1.4291804117502238],
 [33.41180475604646, 40.790284101001006, -1.4571790981839143],
 [33.788002821771606, 40.136013839128, -1.7930938257619253],
 [25.000022199996923, 25.593307217484984, 8.92373021482735]

In [61]:
regi_points_trans_back

[[-41.81985639779847, 17.90003093315857, -0.19289705568297144],
 [49.23404160988244, 15.707299599701548, -0.12278110632563788],
 [6.363323362004561, 1.71499832557424, -16.22729514075901],
 [9.216402345385553, -24.748755949904734, 0.8214513223800974],
 [3.6864067415884834, 20.592287597531286, -13.887971891440138],
 [-33.480910235191374, 32.161728044980464, -25.07667197005128],
 [-33.14633429189386, 32.23553120334702, -25.246154811853355],
 [-33.12314508493692, 32.71610589697311, -25.10994654962374],
 [-33.19953427662368, 32.130182205928335, -25.044037490508327],
 [-33.33408566258631, 32.32248793406807, -25.502290426481043],
 [-33.38932727944582, 32.09625328405763, -25.0080199572947],
 [-33.93083863538276, 32.28084870820389, -25.324985634322076],
 [-33.43103283717423, 32.131790253657954, -25.096110823640466],
 [-33.41180475604645, 32.183827138223045, -25.103225264924383],
 [-33.7880028217716, 31.898548864615606, -24.425342836783084],
 [-25.00002219999692, 13.86954756071479, -23.287034713

In [62]:
regi_data_points

[[-41.81985639779847, 17.90003093315857, -0.19289705568297116],
 [49.23404160988244, 15.70729959970155, -0.12278110632563655],
 [6.36332336200456, 1.7149983255742418, -16.227295140759008],
 [9.216402345385553, -24.748755949904734, 0.8214513223800988],
 [3.686406741588483, 20.59228759753129, -13.887971891440134],
 [-33.48091023519138, 32.16172804498047, -25.076671970051276],
 [-33.146334291893865, 32.23553120334702, -25.246154811853344],
 [-33.12314508493693, 32.71610589697311, -25.109946549623732],
 [-33.19953427662369, 32.130182205928335, -25.044037490508316],
 [-33.33408566258632, 32.32248793406807, -25.502290426481036],
 [-33.38932727944583, 32.09625328405764, -25.008019957294696],
 [-33.93083863538277, 32.28084870820389, -25.32498563432207],
 [-33.431032837174236, 32.131790253657954, -25.09611082364046],
 [-33.41180475604646, 32.18382713822305, -25.103225264924376],
 [-33.788002821771606, 31.898548864615613, -24.42534283678308],
 [-25.00002219999692, 13.869547560714791, -23.2870347